In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload
import profile

In [ ]:
pd.set_option('display.max_columns',100)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda fl: f'{fl:,.2f}')
matplotlib.rcParams['figure.figsize'] = [8.0, 8.0/1.61]

In [ ]:
import tax; reload(tax);
import policies; reload(policies);
import simulate; reload(simulate); from simulate import *

In [ ]:
df = test_yaml('config.yaml')

In [ ]:
nsims = df.Sim.max()

In [ ]:
plt.hist(df.Return[~df.Last],bins=nsims//5, density=True);
ret_mean = df.Return.mean()
ret_std = df.Return[~df.Last].std()
plt.xlabel(f'return (mean {ret_mean:.1f}%, stdev={ret_std:.1f}%)')
plt.ylabel('Density')
plt.grid(True)

In [ ]:
f0 =  df.Surv1 &  df.Surv2 & df.Last
f1 =  df.Surv1 & ~df.Surv2 & df.Last
f2 = ~df.Surv1 &  df.Surv2 & df.Last

for f in [f0, f1, f2]:
    pbankrupt = float(sum(df.TotAssets[f]==0))/sum(df.Last)
    print(100*pbankrupt)

In [ ]:
pbankrupt = float(sum(df.TotAssets[df.Last]==0))/sum(df.Last)
#print(pbankrupt)
plt.hist(df.TotAssets[df.Last&(df.TotAssets>0)].apply(np.log10),bins=nsims//50,density=True);
plt.xlabel(f'$\log_{{10}}$ of assets at death of both persons (prob of bankruptcy={pbankrupt})');
plt.ylabel('density')
qs = [0.25,0.50,0.75]
quantiles = [df.TotAssets[df.Last].quantile(q) for q in qs]
ylims=plt.ylim()
for qv in quantiles:
    plt.plot([np.log10(qv),np.log10(qv)],ylims,'k',linewidth=0.5)

In [ ]:
fig, axs = plt.subplots(1,2)
for ipers in [1,2]:
    ax = axs[ipers-1]
    col_name = f'A{ipers}'
    age = df[col_name]
    nbins = max(age) - min(age) + 1
    ax.hist(age[df.Last],bins=nbins,density=True)
    ax.set_xlabel(f'Age of person {ipers} at death')
    quantiles = [age[df.Last].quantile(q) for q in qs]
    ylims=ax.get_ylim()
    for qv in quantiles:
        ax.plot([qv,qv],ylims,'k',linewidth=0.5)

In [ ]:
#
# find examples where we failed (ran out of money before we both died)
# and then figure out what happened
#
# 1) did we just live too long?
# 2) crappy stock returns?
# 3) paid too much in taxes? (we can start tapping the 401k money at 59 1/2)
#
# Required Minimum Distribution (RMD) - still haven't built this in to the stuff.
#
df_fails = df[(df.TotAssets==0) & df.Last][['Sim']]
df_fails['Fail'] = True
gb = df[~df.Last][['Sim','Withdraw','Taxes','Return','Year']].groupby(by='Sim')
df2 = gb.agg(func={'Withdraw':'sum', 
                   'Taxes':'sum',
                   'Return':np.mean,
                   'Year':len})
df2['AvgTaxPerYear'] = df2['Taxes']/df2['Year']
df2 = df2.merge(df_fails,how='left',on='Sim')
df2.Fail = df2.Fail.apply(lambda xx: True if xx==True else False)

In [ ]:
print(df2.Sim[df2.Fail].tolist())

In [ ]:
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter, FuncFormatter

def plot_sim(df=df[df.Sim==1]):
    df = df[:-1]
    fig, axs = plt.subplots(4,1,figsize=(13,8))
    axs[0].stackplot(df.Year, [df.Cash401k, df.PstTaxCash, df.Stock401k, df.PstTaxStok], labels=['cash401k','cash','stock401k','stock'])
    axs[0].legend()
    axs[0].set_xlim([2022,2078])
    axs[0].yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
    
    axs[1].plot(df.Year, df.Withdraw, label='withdraw')
    axs[1].plot(df.Year, df.Spend, label='spend')
    axs[1].plot(df.Year, df.RMD, label='RMD')
    axs[1].legend()
    axs[1].yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
    axs[1].set_xlim([2022,2078])
    
    axs[2].bar(df.Year, df.SSN, label='Social Security')
    axs[2].bar(df.Year, df.Ins, label='Insurance')
    axs[2].set_xlim([2022,2078])
    axs[2].legend()
    axs[2].yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))

    axs[3].plot(df.Year, df.Taxes, label='Taxes')
    axs[3].set_xlim([2022,2078])
    axs[3].legend()
    axs[3].yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
    
    for yr, surv1 in zip(df.Year, df.Surv1):
        if not surv1:
            axs[0].axvline(yr,color='blue')
            print('1 dies in year', yr)
            break
    for yr, surv2 in zip(df.Year, df.Surv2):
        if not surv2:
            axs[0].axvline(yr,color='red')
            print('2 dies in year', yr)
            break

In [ ]:
plot_sim(df[df.Sim==86])

In [ ]:
df[df.Sim==86]